<h1 style="text-align:center;">Setting the ESKM data for HPE iLO Systems</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This script demonstrates how to <b>set the ESKM (Enterprise Secure Key Manager) username, password, and other related configurations</b> for HPE iLO systems.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">2.</div><br>
    <p>The code below defines a function called ESKM_username_pass that is responsible for configuring the ESKM (Enterprise Secure Key Manager) settings on an HPE iLO system using the Redfish API. The function takes several parameters, including the ESKM username, password, account group, primary key server address, and primary key server port.</p>
<br>The following steps explain this function:<br>

1. Get the resource directory to find the URI for the ESKM service.
2. If the resource directory is not available or disabled, manually search for the relevant URI by navigating through the API hierarchy.
3. If the URI for the ESKM service is found, create a dictionary object with the login name, password, account group, ESKM local CA certificate name, primary key server address, and primary key server port.
4. Use the HTTP PATCH method to update the ESKM service configuration with the dictionary object.
5. Check the response status code to determine if the update was successful or if an error occurred. If there was an error, print the error message or status code. If the update was successful, print the response body.
<br><br>
The ESKM_username_pass function is used to configure the ESKM settings on an HPE iLO system. It retrieves the ESKM URI, populates the configuration data, sends a PATCH request to update the settings, and handles the response accordingly. Upon successful update, it prints a success message along with the response details.
</div>

In [ ]:
def ESKM_username_pass(_redfishobj, eskm_username, eskm_password, eskm_accountgroup, \
                       eskm_primarykeyserver_addr, eskm_primarykeyserver_port):

    eskm_uri = None
    body = dict()

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        security_service_uri = managers_members_response.obj.Oem.Hpe.Links['SecurityService']\
                                                                                    ['@odata.id']
        security_service_response = _redfishobj.get(security_service_uri)
        eskm_uri = security_service_response.obj.Links['ESKM']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeESKM.' in instance['@odata.type']:
                eskm_uri = instance['@odata.id']
                break

    if eskm_uri:
        body["KeyManagerConfig"] = dict()
        body["KeyManagerConfig"]["LoginName"] = eskm_username
        body["KeyManagerConfig"]["Password"] = eskm_password
        body["KeyManagerConfig"]["AccountGroup"] = eskm_accountgroup
        body["KeyManagerConfig"]["ESKMLocalCACertificateName"] = ""
        body["PrimaryKeyServerAddress"] = eskm_primarykeyserver_addr
        body["PrimaryKeyServerPort"] = eskm_primarykeyserver_port
        resp = _redfishobj.patch(eskm_uri, body)
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">3.</div><br>
    
The code below establishes a connection with an iLO server using the specified system URL, login account, and password. It then sets the primary ESKM key server address and port. The script utilizes the Redfish protocol to communicate with the server and performs the necessary operations to configure the key server. Finally, it logs out from the iLO server.
</div>

<div class = "alert alert-block alert-warning" style="border-radius: 20px;">
    <b>When running on the server locally</b> use the following values<br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    ESKM_USERNAME = "admin"
    ESKM_PASSWORD = "password"
    ESKM_ACCOUNTGROUP = "group"
    ESKM_PRIMARYKEYSERVER_ADDR = "192.168.1.10"
    ESKM_PRIMARYKEYSERVER_PORT = 5927
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    ESKM_username_pass(REDFISHOBJ, ESKM_USERNAME, ESKM_PASSWORD, ESKM_ACCOUNTGROUP, \
                       ESKM_PRIMARYKEYSERVER_ADDR, ESKM_PRIMARYKEYSERVER_PORT)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    4.Changing Boot Setting Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/?python#changing-boot-settings">LINK</a><br>
    
</div>